In [45]:
from ipywidgets import IntProgress
import numpy as np
import numpy.random as random
import scipy as sp


In [46]:
def rand(low=0.0, high=1.0, size=None):
    return random.uniform(low=low, high=high, size=size)

In [47]:
a = np.array([1,2,3])

def insert(arr, index, value):
    if index > len(arr):
        new_arr = np.zeros((index + 1,))
        new_arr[:len(arr)] = arr
        arr = new_arr
    arr[index] = value
    return arr
        
insert(a, 10, 7)


array([1., 2., 3., 0., 0., 0., 0., 0., 0., 0., 7.])

In [48]:
np.concatenate(([1], np.arange(5, 50 + 5, 5)))

array([ 1,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50])

In [49]:
np.tile(np.arange(2, 7), 4)

array([2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6, 2, 3, 4, 5, 6])

In [53]:
# define a range of learning rates to test
alphas = np.arange(0.05, 1 + 0.05, 0.05)
# define a range of softmax parameters to test
betas = np.concatenate(([1], np.arange(5, 50 + 5, 5)))
# define a range of WM reliance to test
rhos = np.arange(0, 1 + 0.05, 0.05)
# define a range of capacities to test
Ks = np.arange(2, 6 + 1)

# for a finer grid:
# alphas = [.06:.01:.5];%[0.05:.05:1];
# betas = [1 4:2:20];%[1 5:5:50];
# rhos = [.5:.01:.98];%[0:.05:1];


# define simulation parameters
realalpha = .1
realbeta = 10
realrho = .9
realK = 4
## simulate the RLWM task

b = 0
t = 0


Reps = 3
Trials = 15
nsMin = 2
nsMax = 6

alloc = int(Reps * Trials * (nsMax - nsMin + 1) / 2 * ( 2 * nsMax + (nsMax - nsMin)))
update = np.zeros((alloc,), dtype=int)
choice = np.zeros((alloc,), dtype=int)
rew = np.zeros((alloc,), dtype=int)
stim = np.zeros((alloc,), dtype=int)
setsize = np.zeros((alloc,), dtype=int)

for rep in range(Reps): # three repetitions of each set size
    for ns in range(nsMin, nsMax + 1): # block of set size ns
        
        b = b + 1
        update[t] = 1
        # initialize WM mixture weight
        w = realrho * np.min([1, realK / ns])
        # initialize RL and WM agents
        Q = .5 + np.zeros((ns, 3))
        WM = .5 + np.zeros((ns, 3))
        # define a sequence of trials with 15 iterations of each stimulus
        trials = np.tile(np.arange(1, ns + 1, 1), Trials)
        # loop over trials
        for s in trials:
            
            # RL policy
            softmax1 = np.exp(realbeta * Q[s - 1, :]) / np.sum(np.exp(realbeta * Q[s - 1, :]))
            # WM policy
            softmax2 = np.exp(50 * WM[s - 1, :]) / np.sum(np.exp(50 * WM[s - 1, :]))
            # mixture policy
            pr = (1 - w) * softmax1 + w * softmax2
            # action choice
            r = rand()
            
            if r < pr[0]:
                choice[t] = 1
            elif r < pr[0] + pr[1]:
                choice[t] = 2 
            else:
                choice[t] = 3
           
            # reward correct action (arbitrarily defined here)
            rew[t] = choice[t] == np.remainder(s, 3) + 1
            
            # update RL and WM agents
            Q[s - 1, choice[t] - 1] += realalpha * (rew[t] - Q[s - 1, choice[t] - 1])
            WM[s - 1, choice[t] - 1] = rew[t]
            # store information
            stim[t] = s
            setsize[t] = ns
            t = t + 1
       

In [55]:
## brute force computation of LLH for multiple parameters

bar_max = len(alphas) * len(betas) * len(rhos) * len(Ks) * len(stim)
bar = IntProgress(min=0, max=bar_max)
display(bar)
print(bar_max)
llh = np.zeros((len(alphas), len(betas), len(rhos), len(Ks)))

i1=0
bar_value = 0
for alpha in alphas:
    i1 = i1 + 1
    i2=0
    for beta in betas:
        i2 = i2 + 1
        j1=0
        for rho in rhos:
            j1 = j1 + 1
            j2 = 0
            for K in Ks:
                j2 = j2 + 1
                l=0
                for t in range(len(stim)):
                    s = stim[t]
                    if update[t]:
                        Q = .5 + np.zeros((setsize[t], 3))
                        WM2 = .5 + np.zeros((setsize[t], 3))
                    
                    w = rho * np.min([1, K / setsize[t]])
                    softmax1 = np.exp(beta * Q[s - 1, :]) / np.sum(np.exp(beta * Q[s-1, :]))
                    softmax2 = np.exp(50 * WM[s - 1, :]) / np.sum(np.exp(50 *WM[s-1, :]))
                    pr = (1 - w) * softmax1 + w * softmax2
                    l = l + np.log(pr[choice[t] - 1]);
                    Q[s - 1, choice[t] - 1] = Q[s - 1,choice[t] - 1] + alpha * (rew[t] - Q[s - 1, choice[t] - 1])
                    WM[s - 1, choice[t] - 1] = rew[t]
                    
                    bar_value += 1
                    bar.value = bar_value
                
                llh[i1 - 1,i2 - 1,j1 - 1,j2 - 1] = l

IntProgress(value=0, max=41580000)

41580000


/Users/mwibrow/.virtualenvs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in long_scalars


KeyboardInterrupt: 

In [ ]:
print('hello')